# SQL II: Grouping and Joining

_September 21, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [92]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

In [93]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Raizel","Mitch","Eric"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Mike W","Mitch","Mike R"],
                            "quiz_score":np.random.randint(0,10,3)})

In [4]:
conn = sqlite3.connect('aggregate.db') 
# .db has the ability to store multiple tables
c = conn.cursor()

In [5]:
# insert these datasets into our db
auto.to_sql('auto', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
small_grades.head()

,students,quiz_score
0,Mike W,6
1,Mitch,3
2,Mike R,7


In [11]:
small_quiz.head()

,students,projects,grades
0,Raizel,1,94
1,Mitch,2,96
2,Eric,1,87


In [9]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [14]:
# for the tips df/table, get the total number of man and woman
query = """SELECT sex, count(*) FROM tips GROUP BY sex"""
c.execute(query).fetchall()

[('Female', 87), ('Male', 157)]

In [23]:
# (tips.groupby(['sex'])).sex.value_counts()
(tips.groupby(['sex'])).size()

sex
Male      157
Female     87
dtype: int64

Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [36]:
# using pandas, select the average amount tipped for time
(tips.groupby(['time']))['tip'].mean()

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [27]:
# use sql to recreate this query
query = """SELECT time, avg(tip) FROM tips GROUP BY time"""
c.execute(query).fetchall()

[('Dinner', 3.102670454545454), ('Lunch', 2.7280882352941176)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [37]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [75]:
# use pandas, get the average total bills for a party larger than 2 for every size of party

# using pandas

avg_bill = tips[tips['size'] > 2]
(avg_bill.groupby(['size'])['total_bill'].max()) ## or index this series

size
3    50.81
4    48.33
5    41.19
6    48.17
Name: total_bill, dtype: float64

In [74]:
# use sql to recreate this query
query = """SELECT size, max(total_bill) 
           FROM tips 
           WHERE size > 2 
           GROUP BY size"""
c.execute(query).fetchall()

[(3, 50.81), (4, 48.33), (5, 41.19), (6, 48.17)]

In [77]:
# your turn - use pandas, get the maximum value of 
# total bill for female non smoker

# using group by?

tips[(tips['sex'] == 'Female') & (tips['smoker'] == 'No')].total_bill.max()

35.83

In [89]:
# using group by
tips.groupby(['sex', 'smoker']).total_bill.max()['Female']['No']

35.83

In [76]:
# use sql to recreate this query
query = """SELECT max(total_bill)
           FROM tips
           WHERE sex = 'Female'
           AND smoker = 'No'"""
c.execute(query).fetchall()

[(35.83,)]

In [87]:
# your turn - use pandas, get the average tip 
# values for different size groups, and show top 2
# most amount of average tips


tips.groupby(['size']).tip.mean().sort_values(ascending=False)[:2]

size
6    5.225000
4    4.135405
Name: tip, dtype: float64

In [81]:
# use sql 
query = """SELECT size, avg(tip)
           FROM tips
           GROUP BY size
           ORDER BY avg(tip) DESC
           LIMIT 2"""
c.execute(query).fetchall()

[(6, 5.225), (4, 4.135405405405407)]

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [90]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0   Mike W           6
1    Mitch           3
2   Mike R           7
  students  projects  grades
0   Raizel         1      94
1    Mitch         2      96
2     Eric         1      87


In [95]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, left_on = 'students', right_on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Mike W,8,NaN,NaN
1,Mitch,2,2.0,91.0
2,Mike R,4,NaN,NaN


In [118]:
# sql 
# query = """SELECT * 
#            FROM small_quiz as sq, small_grade as sg
#            WHERE sq.students = sg.students
                 
#            """
query = """SELECT *
           FROM small_grade as sg
           LEFT JOIN small_quiz as sq ON sq.students = sg.students
                 
           """
c.execute(query).fetchall()

[(0, 'Mike W', 6, None, None, None, None),
 (1, 'Mitch', 3, 1, 'Mitch', 2, 96),
 (2, 'Mike R', 7, None, None, None, None)]

In [119]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

,students,quiz_score,projects,grades
0,Mitch,2,2,91


In [120]:
query = """SELECT * FROM small_quiz JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(1, 'Mitch', 2, 96, 1, 'Mitch', 3)]

In [ ]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""